Техническое задание содержится в файле
Документация.rar\ч. V Техничесая часть.docx

# 1. Поиск ТЗ в списке документации к закупке

In [49]:
# pip install bayoo-docx
# pip install aspose-zip
# pip install aspose-words
# pip install pdfplumber #Работа с .PDF

import os, shutil, sys, re
from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import aspose.zip as az
import aspose.words as aw
import pdfplumber
import pandas as pd

def doc2txt(path):
    doc = aw.Document(path).get_text()
    s = re.sub('[\x07\x0b\x13HYPERLINK]', ' ', doc).split('\r')
    return '\n'.join([i.strip() for i in s if len(i.strip())>0][1:-1]) 

def zip2dir(fname):
    # Загрузить архив
    with az.Archive(fname) as archive:
	    # Извлечь файл 
        fn = os.path.basename(fname)
        archive.extract_to_directory(os.path.join(path_temp, fn))

def rar2dir(fname):
    # Загрузить архив
    with az.rar.RarArchive(fname) as archive:
        # Извлечь файл 
        fn = os.path.basename(fname)
        archive.extract_to_directory(os.path.join(path_temp, fn))

def iter_block_items(parent):
    """
    Generate a reference to each paragraph and table child within *parent*,
    in document order. Each returned value is an instance of either Table or
    Paragraph. *parent* would most commonly be a reference to a main
    Document object, but also works for a _Cell object, which itself can
    contain paragraphs and tables.
    """
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    elif isinstance(parent, _Row):
        parent_elm = parent._tr
    else:
        raise ValueError("something's not right")
    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

def docx2txt(docx_file):
    document = Document(docx_file)
    text = []
    for block in iter_block_items(document):
        #print(block.text if isinstance(block, Paragraph) else '<table>')
        if isinstance(block, Paragraph):
            if len(block.text.strip()) > 0:
                text.append(block.text)
        elif isinstance(block, Table):
            for row in block.rows:
                row_data = []
                for cell in row.cells:
                    for paragraph in cell.paragraphs:
                        row_data.append(paragraph.text)
                text.append("\t".join(row_data))    
    return '\n'.join(text)

def pdf2txt(fname):
    with pdfplumber.open(fname) as pdf:
        if len(pdf.pages):
            text_o = ' '.join([
                page.extract_text() or '' for page in pdf.pages if page
            ])
    return text_o

def xlsx2txt(fname):
    return pd.read_excel(fname).to_string()

def to_txt(path):
    txt_from_files = {}
    for root, dirs, files in os.walk(path):
        #print(f'{root=}, {dirs=}, {files=}')
        for fn in files:
            ext = fn.split('.')[-1]
            ffn = os.path.join(root, fn)
            if ext=='doc':
                txt_from_files[ffn] = doc2txt(ffn)
            elif ext=='docx':
                txt_from_files[ffn] = docx2txt(ffn)
            elif ext=='txt':
                with open(ffn,'r') as f:
                    txt_from_files[ffn] = f.read()
            elif ext in [ 'xlsx']:
                txt_from_files[ffn] = xlsx2txt(ffn)
            elif ext=='rar':
                rar2dir(ffn)
            elif ext=='zip':
                zip2dir(ffn)
    return txt_from_files

def clear_temp(path_temp):# Очистка временной папки
    if os.path.exists(path_temp): 
        shutil.rmtree(path_temp)
    os.mkdir(path_temp)

def find_nearest_file(texts, keywords):
    '''
    На вход подается словарь вида имя файла: текст и список ключевых фраз в формате re
    Функция ищет по ключевым фразам в тексте наиболее близкий по содержанию файл
    На выходе имя найденного файла 
    '''
    closest_file = None
    max_count = 0
    for filename, text in texts.items():
        if len(text)>0: 
            if sum([len(re.findall (k, os.path.basename(filename).lower())) for k in keywords]): # Если найдены совпадения в названии файла
                return filename
            else:
                text = re.sub("[^а-я\n ]", "", text.lower()).strip() # оставляем только русские буквы и перевод строки
                cnt = sum([len(re.findall (k, text)) for k in keywords])
                distance = 0
                text = text.split('\n')
                if cnt: # если найдено хоть одно совпадение
                    for i,s in enumerate(text): # то ищем номера строк в которых эти ключи найдены
                        cnt = sum([len(re.findall (k, s)) for k in keywords])
                        if cnt:
                            distance += cnt/(i+1) # вычисляем дистанцию, чем дальше фраза от начала документа, тем меньше ее вес
                    if distance > max_count:
                        max_count = distance
                        closest_file = filename
                print(f'{distance=:0.3f}, {filename=}')
                    
    return closest_file

# ==================================================================
path_root = "C:/temp/"
path_temp = "temp"
keywords = [r"\bтехническ.. задан*",  "описание объекта закупки"]

if  not os.path.exists(path_root):
    print("Папка не существует")
    sys.exit()

dirs = next(os.walk(path_root))[1]
# dirs = ['0124600003224000028']
for path in dirs:
    clear_temp(path_temp)
    path = os.path.join(path_root, path)    
    txt_from_files = to_txt(path)
    txt_from_arch = to_txt(path_temp)
    if txt_from_arch:
        txt_from_files = txt_from_files | txt_from_arch
    #clear_temp(path_temp)
    file_tz = find_nearest_file(txt_from_files, keywords)
    if file_tz:
        print(f'Техническое задание из каталога {path} с наибольшей вероятностью содержится в файле: {file_tz}')
        print(txt_from_files[file_tz][:200])
    else:
        print(f'Техническое задание из каталога {path} не найдено')
    print()

    
for key, value in txt_from_files.items():
    if len(value)!=0:
        fn = os.path.basename(key)
        fn = '.'.join(fn.split('.')[:-1]) +'.txt'
        with open(os.path.join(path_temp, fn), 'w', encoding='utf-8') as f:
            f.write(value)


Техническое задание из каталога C:/temp/0123200000324000309 с наибольшей вероятностью содержится в файле: C:/temp/0123200000324000309\docs\Описание объекта закупки.docx.docx
Техническое задание 
на оказание услуг по сопровождению государственной информационной системы обеспечения градостроительной деятельности Амурской области
1. ОБЩИЕ СВЕДЕНИЯ
1.1. Описание услуг
1.1.1. 

distance=0.000, filename='C:/temp/0124200000624000686\\docs\\Обоснование начальной (максимальной) цены контракта.docx'
Техническое задание из каталога C:/temp/0124200000624000686 с наибольшей вероятностью содержится в файле: C:/temp/0124200000624000686\docs\Описание объекта закупки.docx
Описание объекта закупки в соответствии со статьей 33 Федерального закона от 05 апреля 2013 года № 44-ФЗ «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и 

distance=0.000, filename='C:/temp/0124200000624000712\\docs\\Обоснование начальной (максимальной) цены контракта.docx'
Техническое зад

In [60]:
import re

def search_keys(text, keys):
    '''
    Функция считает количество ключевых слов/фраз в тексте
    На входе - текст, список ключевых слов в виде списка фраз в формате regex
    На выходе - словарь с ключами и их количеством
    '''
    text =  text.lower().strip() # переводим в нижний регистр
    cnt= {l[0]:len(l) for k in keys if len(l:=re.findall(k, text))>0}       
    return cnt


neg_keys = [
    r"континент",
    r"техническая поддержка",
    r"абонемент",
    r"доработка",
    r"защита (информации)",
    r"консультации",
    r"консультационные услуги",
    r"образование",
    r"образовательные услуги",
    r"глонасс",
    r"gps",
    r"бюджет",
    r"антивирус",
    r"контур контур контур",
    r"(?:\bмис\b|медицинская информационная система)",
    r"мис ариадна",
    r"техэксперт",
    r"бд супа",
    r"технокад",
    r"omnicommonline",
    r"карант",
    r"парус",
    r"континент",
    r"оборудование",
    r"лицензия",
    r"неисключительные права",
    r"аттестация",
    r"контур-фокус",
    r"сертификат",
    r"брест",
    r"vipnet",
    r"барс-здравоохранение",
    r"бит.управление медицинскими сервисами",
    r"autosat",
    r"varonis",
    r"гранд-смета",
    r"спарк",
    r"кодекс",
    r"иас контрагент",
    r"катарсис",
    r"статус-поставщик",
    r"rosa",
    r"labforce",
    r"enterprize",
    r"vizualization",
    r"businessstudio",
    r"право использования"
    ]
pos_keys = [
    r"(?:созда|разработ)[а-я]{2,3} сайт\w*",
    r"(?:созда|разработ)[а-я]{2,3} мобильн[а-я]{2,3} приложени\w*",
    r"(?:созда|разработ)[а-я]{2,3} пользовательск[а-я]{2,3} интерфейс\w*",
    r"(?:созда|разработ)[а-я]{2,3} нейросет\w*",
    r"(?:созда|разработ)[а-я]{2,3} алгоритм\w* анализа данных",
    r"(?:созда|разработ)[а-я]{2,3} архитектур\w* приложен\w*",
    r"(?:созда|разработ)[а-я]{2,3} дашборд\w*",
    r"(?:созда|разработ)[а-я]{2,3} микросервис\w*",
    r"(?:созда|разработ)[а-я]{2,3} механизм\w* аутентификац\w*",
    r"интегрирова[а-я]{2,3} внешн[а-я]{2,3} сервис\w*",
    r"интегрирова[а-я]{2,3} систем\w? аналитики и мониторинг\w?",
    r"автоматиз[а-я]{4,7} бизнес-процесс",
    r"провести тестирование производительности и безопасности",
    r"оптимиз[а-я]{4,7} код\w*",
    r"оптимиз[а-я]{4,7} интерфейс\w*",
    r"оптимиз[а-я]{4,7} запрос\w*",
    r"развер[а-я]{4,7} приложени\w на сервер\w*",
    r"провести аудит и рефакторинг кода",
    r"емейл-маркетинг",
    r"автоматизация процессов",
    r"сопровождение портала",
    r"актуализация документации",
    r"информационное сопровождение",
    r"smm-сопровождение",
    r"обработка данных",
    r"размещение материалов",
    r"реклама",
    r"размещение информации",
    r"предоставление хостинга",
    r"мониторинг сми",
    r"контент",
    r"маркетинговые услуги",
    r"продвижение в социальных сетях",
    r"базы данных",
    r"аналитическое сопровождение",
    r"мониторинг",
    r"контент-маркетинг",
    r"веб-разработка",
    r"рекламные услуги",
    r"информационное обслуживание",
    r"информационная кампания",
    r"облачные сервисы",
    r"электронная база данных",
    r"мультимедийные услуги",
    r"анализ данных",
    r"сетевое издание",
    r"цифровой образовательный ресурс",
    r"модернизация сайтов",
    r"онлайн-продажи",
    r"искусственный интеллект",
    r"портал",
    r"автоматизация",
    r"сервис",
    r"информационная система",
    r"агрегатор",
    r"личный кабинет",
    r"онлайн",
    r"анализ",
    r"логотип",
    r"лэндинг сайт",
    r"ведение соцсетей",
    r"интернет магазин",
    r"промо-сайт",
    r"интеграция",
    r"сервис",
    r"интернет ресурс",
    r"естественная речь",
    r"естественный язык",
    r"машинное обучение",
    r"\bml\b",
    r"\bai\b",
    r"\bcrm\b",
    r"биг-дата",
    r"бигдата",
    r"интернет",
    r"компьютерное зрение",
    r"devops",
    r"данные",
    r"разработка",
    r"приложение",
    r"фронтенд",
    r"бэкенд",
    r"маркетинг",
    r"веб-сервисы",
    r"веб-аналитика"
    ]

with open(r'C:\Users\user\Documents\python\1t\temp\ч. V Техничесая часть.txt', 'r', encoding='utf-8') as f:
    text = f.read()
pos_cnt = search_keys(text, pos_keys)
neg_cnt = search_keys(text, neg_keys)
print(f'Всего слов-маркеров {sum(pos_cnt.values())}:')
print(*[f'{k}({v})' for k,v in pos_cnt.items()], sep=', ')
print(f'Всего стоп-слов {sum(neg_cnt.values())}:')
print(*[f'{k}({v})' for k,v in neg_cnt.items()], sep=', ')

Всего слов-маркеров 105:
автоматизирующее бизнес-процесс(1), контент(1), мониторинг(1), портал(6), сервис(65), информационная система(1), личный кабинет(3), онлайн(1), анализ(8), интеграция(3), ai(1), интернет(8), данные(2), разработка(3), приложение(1)
Всего стоп-слов 6:
техническая поддержка(1), консультации(1), образование(1), антивирус(1), лицензия(1), сертификат(1)
